In [210]:
# add autoreload
%load_ext autoreload
%autoreload 2
import sickle
from tqdm import tqdm
import os
import requests
import lxml
import bs4
import random
from time import sleep


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [211]:
tlist = ['cardiovascular', 'cardiogram', 'cardiology', 'heart', 'vascular', 'angiogram', 'cardiologie', 'hartziekte', 'vaatziekte', 'medical', 'clinical', 'surgical']
institute = 'VU'
base_url =   'https://research.vu.nl/ws/oai?metadataPrefix=oai_dc' #'https://repository.ubn.ru.nl/oai/openaire'  https://scholarlypublications.universiteitleiden.nl/oai2, http://dspace.library.uu.nl/oai/dissertation
pdf_path = '//Ds/data/LAB/laupodteam/AIOS/Bram/language_modeling/MEDICAL_TEXT/RAW/PhDTheses'

In [218]:
OpenAIRE_institutes = ['VU']

In [212]:
sickler = sickle.Sickle(base_url)


In [213]:
sets = sickler.ListSets()

In [214]:
Sets = {}
for s in sets:
    Sets[s.setSpec]  = s.setName    

In [226]:
keywords = ['medisch', 'medical', 'diss', 'phd', 'thesis']
if institute in OpenAIRE_institutes:
    keywords = keywords + ['publications:withfiles']

Sets_to_mine = []
for key, val in Sets.items():
    if any([c in val.lower() for c in keywords]) | any([c in key.lower() for c in keywords]):
        print(f'Set: {key} contains keyword')
        Sets_to_mine.append(key)

Set: publications:withFiles contains keyword


In [229]:
# get records 
from collections import defaultdict
records_lists = defaultdict(list)
for set_to_mine in tqdm(Sets_to_mine):
    records = sickler.ListRecords(metadataPrefix='oai_dc', 
                                  ignore_deleted=True, 
                                  set=set_to_mine) # dissertation com_1874_298213
    for record in records:
        records_lists[set_to_mine].append(record)

  0%|          | 0/1 [00:02<?, ?it/s]


BadArgument: The request includes illegal arguments or is missing required arguments. resumptionToken is exclusive!

In [192]:
link_list = []
error_list = []
for set_to_mine in Sets_to_mine:
    relevant_counter = 0
    for r in tqdm(records_lists[set_to_mine]):
        meta = r.get_metadata()
        relevant = False
        
        try:
            if any([t in subj for subj in meta['subject'] for t in tlist]):
                relevant = True
        except:
            pass

        try:
            if any([t in subj for subj in meta['title'] for t in tlist]):
                relevant = True
        except:
            pass

        try:
            if any([t in subj for subj in meta['description'] for t in tlist]):
                relevant = True
        except:
            pass 
        
        try:
            if ('doctoral' in meta['type'][0].lower()) & relevant:
                relevant = True
            else:
                relevant = False
        except:
            pass

        try:
            if ('embargo' in meta['rights'][0].lower()) & relevant:
                relevant = False
        except:
            pass  
        
        if relevant:
            relevant_counter += 1
            if institute == 'LUMC':
                try:
                    link = meta['identifier'][-1] 
                    # identify first url in list
                    found_link = False
                    for l in meta['identifier']:
                        if ('http:' in l) or ('https:' in l):
                            link = l
                            found_link = True
                            break
                    if not found_link:
                        error_list.append(f'No link found for {meta["identifier"]}')  

                    doc_id = link.split('/')[-1]
                    doc_id_int = int(doc_id)+2

                    link = f"https://scholarlypublications.universiteitleiden.nl/handle/1887/{doc_id}"
                    linkPdf = f"https://scholarlypublications.universiteitleiden.nl/access/item%3A{doc_id_int}/download"

                    # extract through link url. The directory can be found in <li class='ubl-file-download'> <a href='...'>
                    # only if <a href in ubl-file-view is "full"
                    r = requests.get(link)
                    # random number between 0.5 and 2.5 seconds
                    rndSleep = round(random.uniform(0.5, 2.5), 2)
                    sleep(rndSleep)
                    soup = bs4.BeautifulSoup(r.text, 'html.parser')
                    found, dsfound, esfound = False, False, False
                    for _res in soup.findAll('li', {'class':'ubl-file-view'}):
                        if _res.a is not None:
                            if _res.a.contents[0].strip().lower() == 'full text':
                                _pdfdir = _res.a['href']
                                found = True
                            elif _res.a.contents[0].strip().lower() == 'summary in dutch':
                                _dutch_summary = _res.a['href']
                                dsfound = True
                            elif _res.a.contents[0].strip().lower() == 'summary in english':
                                _english_summary = _res.a['href']
                                esfound = True

                    linkPdfAlt = f"https://scholarlypublications.universiteitleiden.nl{_pdfdir}" if found else None
                    DutchSummaryLink = f"https://scholarlypublications.universiteitleiden.nl{_dutch_summary}" if dsfound else None
                    EnglishSummaryLink = f"https://scholarlypublications.universiteitleiden.nl{_english_summary}" if esfound else None
                    
                    try:
                        lang = meta['language'][0]
                    except:
                        lang = None
                except Exception as e:
                    error_list.append(f'Error: {e} for link: {link}, with meta data: {meta["identifier"]}')
                    pass
                    #raise ValueError(f'Could not find pdf link for {link}, with error raised: {e}')
            elif institute == 'UU':                
                link = meta['identifier'][0]
                baselink = link.replace('dspace.library.uu.nl/', 'dspace.library.uu.nl/bitstream/')
                linkPdf = baselink + '/full.pdf'

                try:
                    linkPdfAlt = baselink +'/'+meta['creator'][0].split(',')[0].lower()+'.pdf'
                except:
                    linkPdfAlt = None
                
                DutchSummaryLink = None
                EnglishSummaryLink = None
                try:
                    lang = meta['language'][0]
                except:
                    lang = None

            elif institute == 'Radboud':
                try:
                    found_link = False
                    found_pdf_link = False
                    for l in meta['identifier']:
                        if l is not None:
                            if (('http:' in l) or ('https:' in l)) & (l.split("/")[-1].endswith('.pdf')):
                                linkPdf = l
                                found_pdf_link = True                            
                            elif (('http:' in l) or ('https:' in l)):
                                link = l
                                found_link = True                           

                    if not found_link:                        
                        error_list.append(f'No link found for {meta["identifier"]}')  
                        continue
                    if not found_pdf_link:
                        error_list.append(f'No pdf link found for {meta["identifier"]}')
                        linkPdf = None

                    linkPdfAlt = None
                    DutchSummaryLink = None
                    EnglishSummaryLink = None
                    try:
                        lang = meta['language'][0]
                    except:
                        lang = None
                except Exception as e:
                    error_list.append(f'Error: {e} for link: {link}, with meta data: {meta["identifier"]}')
                    continue
                    #raise ValueError(f'Could not find pdf link for {link}, with error raised: {e}')                

            try:
                link_list.append({'Set':set_to_mine, 
                                'Link': link, 
                                'PdfLink': linkPdf,
                                'PdfLinkAlt': linkPdfAlt,
                                'DutchSummaryLink': DutchSummaryLink,
                                'EnglishSummaryLink': EnglishSummaryLink,
                                'Title': meta['title'][0],
                                'Description': meta['description'][0],
                                'Date': meta['date'][0],
                                'Language': lang,
                                }
                                )
            except Exception as e:
                link_list.append({'Set':set_to_mine, 
                                'Link': link, 
                                'PdfLink': linkPdf,                                
                                'PdfLinkAlt': linkPdfAlt,
                                'DutchSummaryLink': None,
                                'EnglishSummaryLink': None,
                                'Title': None,
                                'Description': None,
                                'Date': None,
                                'Language': None,
                                }
                                )
                error_list.append(f'Error: {e} for link: {link}')
    print(f'Found {relevant_counter} relevant records in set: {set_to_mine}')      

  0%|          | 0/13152 [00:00<?, ?it/s]

100%|██████████| 13152/13152 [00:01<00:00, 8204.56it/s]


Found 823 relevant records in set: col_2066_119645


100%|██████████| 88630/88630 [00:08<00:00, 9898.30it/s] 


Found 704 relevant records in set: col_2066_119636


100%|██████████| 1/1 [00:00<00:00, 2010.69it/s]

Found 0 relevant records in set: col_2066_69015


In [195]:
# remove duplicate entries, based on the title
title_set = set()
unique_link_list = []
for el in link_list:
    try:
        title = el['Title']
        if title not in title_set:
            unique_link_list.append(el)
            title_set.add(title)
    except:
        pass

In [196]:
len(unique_link_list), len(link_list)

(803, 1527)

In [209]:
# now we parse the link list and download the pdfs
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.107 Safari/537.36' }

# add sleep

def pdf_writer(pdf_url, _pdf_path):
    r = requests.get(pdf_url, stream=True)
    if r.status_code == 200:
        with open(_pdf_path, 'wb') as f:
            for i,chunk in enumerate(r.iter_content(chunk_size=1024)): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
        return True, 200
    else:
        return False, r.status_code

pdf_error_list = []
skipped_list = []
src_list = []
rcode_list = []
extract_full_text = True
for link in tqdm(unique_link_list):
    # MAIN PDF
    pdf_name = f"{institute}_{link['Link'].split('/')[-1]}.pdf"
    _pdf_path = os.path.join(pdf_path, pdf_name)
    return_code = None
    if extract_full_text:
        pdf_url = link['PdfLink']
        # check if pdf already exists
        if os.path.isfile(_pdf_path):
            skipped_list.append(f'Pdf already exists: {_pdf_path}')
        else:
            # try to download the pdf
            try:
                write_file, return_code = pdf_writer(pdf_url, _pdf_path)
                if write_file:
                    src_list.append(f'Pdf downloaded from original link: {pdf_url}')            
                elif (not write_file) & (link['PdfLinkAlt'] is not None):
                    pdf_url = link['PdfLinkAlt']
                    write_file, return_code = pdf_writer(pdf_url, _pdf_path)

                    if write_file:                
                        src_list.append(f'Pdf downloaded from alternative link: {pdf_url}')
                    else:
                        pdf_error_list.append(f'Error: {return_code}. No success for both : {link["PdfLinkAlt"]} and {link["PdfLink"]}')
                else:
                    pdf_error_list.append(f'Error: {return_code}. No success for {link["PdfLink"]} and PdfLinkAlt was missing')
            except Exception as e:
                pdf_error_list.append(f'Error: {e} for link: {link["Link"]}')  
            if return_code == 429:
                print(f'Rate limit exceeded...sleeping 30 seconds')
                sleep(30)
                continue
            else:
                sleep(4)

    if link['DutchSummaryLink'] is not None:
        pdf_url = link['DutchSummaryLink']
        _pdf_path_ds = _pdf_path.replace('.pdf', '_dutch_summary.pdf')
        if os.path.isfile(_pdf_path):
            skipped_list.append(f'Pdf already exists: {_pdf_path_ds}')
        else:
            write_file, return_code = pdf_writer(pdf_url, _pdf_path_ds)
            
            if not write_file:
                pdf_error_list.append(f'Error: {return_code}. No success for dutch summary: {pdf_url}')

    if link['EnglishSummaryLink'] is not None:
        pdf_url = link['EnglishSummaryLink']
        _pdf_path_es = _pdf_path.replace('.pdf', '_english_summary.pdf')
        if os.path.isfile(_pdf_path_es):
            skipped_list.append(f'Pdf already exists: {_pdf_path_es}')
        else:
            write_file, return_code = pdf_writer(pdf_url, _pdf_path_es)
            if not write_file:
                pdf_error_list.append(f'Error: {return_code}. No success for english summary: {pdf_url}')    
    
print(f'Found {len(pdf_error_list)} errors while downloading pdfs')
print(f'Skipped {len(skipped_list)} pdfs because they already existed')

 19%|█▉        | 155/803 [09:49<51:42,  4.79s/it]  

Rate limit exceeded...sleeping 30 seconds


 33%|███▎      | 269/803 [23:13<56:27,  6.34s/it]  

Rate limit exceeded...sleeping 30 seconds


 48%|████▊     | 384/803 [35:50<41:49,  5.99s/it]  

...

Rate limit exceeded...sleeping 30 seconds


 87%|████████▋ | 699/803 [1:11:33<08:55,  5.15s/it]

Rate limit exceeded...sleeping 30 seconds


100%|██████████| 803/803 [1:18:51<00:00,  5.89s/it]

Found 116 errors while downloading pdfs
Skipped 106 pdfs because they already existed
